In [1]:
import os

In [2]:
%pwd

'/Users/c5367983/Desktop/Projects/QuantileX/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/c5367983/Desktop/Projects/QuantileX'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    assets_type:str
    start_date:str
    interval:str


In [6]:
from src.constants import *
from src.utils.common import read_yaml, create_directories
from datetime import datetime, timedelta

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
            ):
        self.config= read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir + "/" + config.assets_type])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            assets_type = config.assets_type,
            start_date = config.start_date,
            interval = config.interval
            )

        return data_ingestion_config
    


In [8]:
from datetime import datetime, timedelta
from ta import add_all_ta_features
import yfinance as yf
import pandas as pd
import numpy as np
import os

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        self.df = None
        self.file_path = self.config.root_dir + "/" + self.config.assets_type + "/" + self.config.assets_type + "_" + self.config.start_date + "_" +datetime.now().strftime('%Y-%m-%d') + ".csv"
        
    def download_file(self):
        if not os.path.exists(self.file_path):
            ticker_obj = yf.Ticker(self.config.assets_type)
            start_date = datetime.strptime(self.config.start_date, '%Y-%m-%d')
            end_date = datetime.now().strftime('%Y-%m-%d')
            df = ticker_obj.history(start = start_date, end=end_date, interval=self.config.interval)
            self.df = df
    def save_data(self):
        if not os.path.exists(self.file_path):
            self.df.to_csv(self.file_path, index=True)

 

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.save_data()
except Exception as e:
    raise e    

    


[2024-02-25 20:20:02,361: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-25 20:20:02,362: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-25 20:20:02,363: INFO: common: Created directory at: artifacts]
[2024-02-25 20:20:02,364: INFO: common: Created directory at: artifacts/data_ingestion/BTC-USD]


/Users/c5367983/anaconda3/envs/quantilex/lib/python3.12/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
